## formação de precisão mista

Este módulo permite que os passes para a frente e para trás de sua rede neural a ser feito em FP16 (também conhecido como * metade precisão *). Isto é particularmente importante se você tiver uma GPU NVIDIA com [tensor cores](https://www.nvidia.com/en-us/data-center/tensorcore/), uma vez que pode acelerar o seu treinamento em 200% ou mais.

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.callbacks.fp16 import *
from fastai.vision import *

## visão global

Para treinar o seu modelo em precisão mista você só tem que chamar [`Learner.to_fp16`](/train.html#to_fp16), que converte o modelo e modifica o [`Learner`](/basic_train.html#Learner) existente para adicionar [`MixedPrecision`](/callbacks.fp16.html#MixedPrecision).

In [ ]:
show_doc(Learner.to_fp16)

<h4 id="to_fp16" class="doc_header"><code>to_fp16</code><a href="https://github.com/fastai/fastai/blob/master/fastai/train.py#L43" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#to_fp16-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>to_fp16</code>(**`learn`**:[`Learner`](/basic_train.html#Learner), **`loss_scale`**:`float`=***`None`***, **`max_noskip`**:`int`=***`1000`***, **`dynamic`**:`bool`=***`True`***, **`clip`**:`float`=***`None`***, **`flat_master`**:`bool`=***`False`***, **`max_scale`**:`float`=***`16777216`***, **`loss_fp32`**:`bool`=***`True`***) → [`Learner`](/basic_train.html#Learner)

<div class="collapse" id="to_fp16-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#to_fp16-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>to_fp16</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Put `learn` in FP16 precision mode.  

Por exemplo:

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)
model = simple_cnn((3,16,16,2))
learn = Learner(data, model, metrics=[accuracy]).to_fp16()
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy
1,0.139469,0.115246,0.963199


Os detalhes sobre a formação de precisão mista estão disponíveis em [NVIDIA's documentation](https://docs.nvidia.com/deeplearning/sdk/mixed-precision-training/index.html). Vamos apenas resumir o básico aqui.
O único parâmetro que você pode querer ajustar é `loss_scale`. Isto é usado para dimensionar a perda, de modo que ele não estouro negativo FP16, levando à perda de precisão (esta é revertida para o cálculo final gradiente depois de converter de volta para fp32). Geralmente o padrão `512` funciona bem, no entanto. Você também pode ativar ou desativar o achatamento do parâmetro tensor principal com `flat_master = true`, no entanto, em nossa testar o diferente é insignificante.
Internamente, a chamada de retorno assegura que todos os parâmetros do modelo (excepto camadas batchnorm, que requerem fp32) são convertidos para FP16, e uma cópia fp32 também é guardado. A cópia fp32 (os `parâmetros master`) é o que é usado para realmente atualizar com o otimizador; os parâmetros FP16 são utilizados para calcular os gradientes. Isso ajuda a evitar underflow com taxas de aprendizagem pequenas.
Tudo isso é implementado pela seguinte chamada de retorno.

In [ ]:
show_doc(MixedPrecision)

<h2 id="MixedPrecision" class="doc_header"><code>class</code> <code>MixedPrecision</code><a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/fp16.py#L64" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MixedPrecision-pytest" style="float:right; padding-right:10px">[test]</a></h2>

> <code>MixedPrecision</code>(**`learn`**:[`Learner`](/basic_train.html#Learner), **`loss_scale`**:`float`=***`None`***, **`max_noskip`**:`int`=***`1000`***, **`dynamic`**:`bool`=***`True`***, **`clip`**:`float`=***`None`***, **`flat_master`**:`bool`=***`False`***, **`max_scale`**:`float`=***`16777216`***, **`loss_fp32`**:`bool`=***`True`***) :: [`LearnerCallback`](/basic_train.html#LearnerCallback)

<div class="collapse" id="MixedPrecision-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MixedPrecision-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>MixedPrecision</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Base class for creating callbacks for a [`Learner`](/basic_train.html#Learner).  

### métodos de retorno de chamada

Você não tem que chamar as seguintes funções a si mesmo - eles são chamados pelo sistema [`Callback`](/callback.html#Callback) das fastai automaticamente para ativar a funcionalidade da classe.

In [ ]:
show_doc(MixedPrecision.on_backward_begin)

<h4 id="MixedPrecision.on_backward_begin" class="doc_header"><code>on_backward_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/fp16.py#L93" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MixedPrecision-on_backward_begin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_backward_begin</code>(**`last_loss`**:`Rank0Tensor`, **\*\*`kwargs`**:`Any`) → `Rank0Tensor`

<div class="collapse" id="MixedPrecision-on_backward_begin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MixedPrecision-on_backward_begin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_backward_begin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Scale gradients up by `self.loss_scale` to prevent underflow.  

In [ ]:
show_doc(MixedPrecision.on_backward_end)

<h4 id="MixedPrecision.on_backward_end" class="doc_header"><code>on_backward_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/fp16.py#L99" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MixedPrecision-on_backward_end-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_backward_end</code>(**\*\*`kwargs`**:`Any`)

<div class="collapse" id="MixedPrecision-on_backward_end-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MixedPrecision-on_backward_end-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_backward_end</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Convert the gradients back to FP32 and divide them by the scale.  

In [ ]:
show_doc(MixedPrecision.on_loss_begin)

<h4 id="MixedPrecision.on_loss_begin" class="doc_header"><code>on_loss_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/fp16.py#L89" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MixedPrecision-on_loss_begin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_loss_begin</code>(**`last_output`**:`Tensor`, **\*\*`kwargs`**:`Any`) → `Tensor`

<div class="collapse" id="MixedPrecision-on_loss_begin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MixedPrecision-on_loss_begin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_loss_begin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Convert half precision output to FP32 to avoid reduction overflow.  

In [ ]:
show_doc(MixedPrecision.on_step_end)

<h4 id="MixedPrecision.on_step_end" class="doc_header"><code>on_step_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/fp16.py#L118" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MixedPrecision-on_step_end-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_step_end</code>(**\*\*`kwargs`**:`Any`)

<div class="collapse" id="MixedPrecision-on_step_end-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MixedPrecision-on_step_end-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_step_end</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Update the params from master to model and zero grad.  

In [ ]:
show_doc(MixedPrecision.on_train_begin)

<h4 id="MixedPrecision.on_train_begin" class="doc_header"><code>on_train_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/fp16.py#L77" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MixedPrecision-on_train_begin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_train_begin</code>(**\*\*`kwargs`**:`Any`)

<div class="collapse" id="MixedPrecision-on_train_begin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MixedPrecision-on_train_begin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_train_begin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Prepare the master model.  